In [1]:
import time
import pandas as pd
import numpy as np
import re
import multiprocessing

from MS2ORF import MS2ORF

pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 400)

<H1>Pre-BLAST data analysis</H1>
<H4>Integrate raw .mgf files, Metamorpheus output, and Casanovo output</H4>
<H4>Create BLAST query file for peptide-genome search</H4>

In [8]:
start = time.time()

#-------------------------
#-----user input variables
#-------------------------

input_mgf_file_rawconverter1 = 'raw_data/'
input_mgf_file_msconvert1 = 'raw_data/'
metamorpheus_allPSMs_file1 = 'raw_data/AllPSMs.psmtsv'
metamorpheus_qvalue_cutoff1 = 0.1
casanovo_output_msconvert1 = 'raw_data/'
casanovo_output_rawconverter1 = 'raw_data/'
all_dataframe_savefile1 = 'processed_data/casanovo_compare1.csv'
casanovo_PTM_list1 = ['+57.021', '+15.995', '+0.984']
casanovo_PTM_score_threshold_for_removal1 = 90

blast_query_file_name1 = 'processed_data/blastfile1.fasta'
blast_query_file_condensed_name1 = 'processed_data/blastfile1_condensed.fasta'

whole_genome_DB1 = 'raw_data/'
transcriptome_proteome_DB1 = 'raw_data/'
output_tBLASTn_file1 = 'processed_data/blastfile1_condensed_out.csv'
output_BLASTp_file1 = 'processed_data/blastfile1_protdb_condensed_out.csv'

#-------------------------
#-------------------------

#-----read .mgf files into dataframes
df1 = MS2ORF.read_mgf_headers(input_mgf_file_rawconverter1, 'RC')
df2 = MS2ORF.read_mgf_headers(input_mgf_file_msconvert1, 'MSC')

#-----merge .mgf dataframes
df3 = pd.merge(df1, df2, on='spectrum_ID', how='outer')
df3 = df3[['scan_RC', 'spectrum_ID', 'retention_RC', 'charge_RC', 'precursor_RC', 'retention_MSC', 'charge_MSC', 'precursor_MSC']]
df3.columns = ['scan', 'spectrum_ID', 'retention_RC', 'charge_RC', 'precursor_RC', 'retention_MSC', 'charge_MSC', 'precursor_MSC']

#-----read morpheus file into dataframe
df4 = MS2ORF.read_Morpheus(metamorpheus_allPSMs_file1, metamorpheus_qvalue_cutoff1)

#-----merge .mgf and morpheus dataframes
df5 = pd.merge(df3, df4, on='scan', how='outer').fillna(0)

#-----read Casanovo output files into dataframes, then merge into single dataframe
df6 = MS2ORF.read_Casanovo(casanovo_output_msconvert1, 'MSC')
df7 = MS2ORF.read_Casanovo(casanovo_output_rawconverter1, 'RC')
df8 = pd.merge(df6, df7, on='spectrum_ID', how='outer').fillna(0)

#-----merge .mgf and morpheus dataframes with casanovo dataframes
df9 = pd.merge (df5, df8, on='spectrum_ID', how='outer').fillna(0)

#-----reformat dataframe for sequence comparison
df10 = MS2ORF.reformat_Columns_For_Comparison(df9, ['denovo_seq_MSC', 'denovo_seq_RC', 'sequence'])

#-----compare sequences pairwise
df11 = MS2ORF.compare_Sequences(df10)

#-----save dataframe to external file
df11.to_csv(all_dataframe_savefile1, index=False)

#-----read in dataframe csv
df12 = pd.read_csv(all_dataframe_savefile1)

#-----reformat casanovo amino acid scores (*100, then turn into integer)
df13 = MS2ORF.reformat_aa_Scores(df12, ['aa_scores_RC', 'aa_scores_MSC'])

#-----remove PTM notation and mask denovo sequences below a score threshold
df14 = MS2ORF.mask_denovo_sequence(df13, casanovo_PTM_list1, casanovo_PTM_score_threshold_for_removal1)

#-----create blast fasta file
MS2ORF.create_BLAST_query_file(df14, ['MSC', 'RC'], blast_query_file_name1)

#-----reduce size of blast file by condensing redundant peptide queries
#-----following function produces dictionary that connects peptides to their parent spectra/table entries and also outputs a reformatted fasta for BLAST
peptide_description_dictionary1, seqnum_peptide_dictionary1 = MS2ORF.reduce_Blast_file(blast_query_file_name1, blast_query_file_condensed_name1)

#-----command for running tblastn in multithread mode using 12 threads (uncomment ! lines to enable BLAST from this notebook)
#-----note that BLAST databases need to be built prior to running these commands [see BLAST documentation]
#---genome
#!tblastn -db whole_genome_DB1 -query blast_query_file_condensed_name1 -matrix IDENTITY -num_alignments 5 -outfmt 10 -num_threads 12 -mt_mode 1 -out output_tBLASTn_file1
#---proteome/transcriptome
#!blastp -db transcriptome_proteome_DB1 -query blast_query_file_condensed_name1 -matrix IDENTITY -num_alignments 5 -outfmt 10 -num_threads 12 -mt_mode 1 -out output_BLASTp_file1


end = time.time()
print(end-start)

/Users/peter/MS2ORF/MS2ORF/MS2ORF.py:94: DtypeWarning: Columns (11,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df1 = pd.read_csv(filename1, sep='\t')
/var/folders/b5/prg17fj579zd64gy82n9frm00000gn/T/ipykernel_43093/676498515.py:55: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df12 = pd.read_csv(all_dataframe_savefile1)
/Users/peter/MS2ORF/MS2ORF/MS2ORF.py:333: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp_concat_sequences = pd.Series([])
/Users/peter/MS2ORF/MS2ORF/MS2ORF.py:334: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp_all_concat_sequences = pd.Series([])


780.1210589408875


<H1>Post-BLAST data analysis</H1>
<H4>integrate BLAST results</H4>

In [10]:
start = time.time()

#-------------------------
#-----user input variables
#-------------------------

output_tBLASTn_file1 = 'processed_data/blastfile1_condensed_out.csv'
output_BLASTp_file1 = 'processed_data/blastfile1_protdb_condensed_out.csv'

#-------------------------
#-------------------------

#-----read tBLASTn csv into dataframe
df15 = pd.read_csv(output_tBLASTn_file1, sep=',', names=['seq_num', 'contig', 'positives_percent', 'match_length', 'mismatch_count', 'gaps', 'query_match_start', 'query_match_stop', 'target_start', 'target_stop', 'evalue', 'score'])

#-----read BLASTp csv into dataframe
df16 = pd.read_csv(output_BLASTp_file1, sep=',', names=['seq_num', 'contig', 'positives_percent', 'match_length', 'mismatch_count', 'gaps', 'query_match_start', 'query_match_stop', 'target_start', 'target_stop', 'evalue', 'score'])

#-----assign peptides to seq_names in blast result dataframes
#---df15
pep_names_col1 = []
for row1 in df15.itertuples():
    pep_names_col1.append(seqnum_peptide_dictionary1[row1.seq_num])
df15['sequence'] = pep_names_col1
#---df16
pep_names_col1 = []
for row1 in df16.itertuples():
    pep_names_col1.append(seqnum_peptide_dictionary1[row1.seq_num])
df16['sequence'] = pep_names_col1

#-----groupby peptide 'sequence' column, then select rows with smallest evalue
#---tblastn dataframe
df17 = df15.loc[df15.groupby('sequence').evalue.idxmin()].reset_index(drop=True)
df17 = df17[['sequence', 'contig', 'positives_percent', 'match_length', 'mismatch_count', 'gaps', 'query_match_start', 'query_match_stop', 'target_start', 'target_stop', 'evalue', 'score']]
#---blastp dataframe
df18 = df16.loc[df16.groupby('sequence').evalue.idxmin()].reset_index(drop=True)
df18 = df18[['sequence', 'contig', 'positives_percent', 'match_length', 'mismatch_count', 'gaps', 'query_match_start', 'query_match_stop', 'target_start', 'target_stop', 'evalue', 'score']]

#----assign e-values to main combined dataframe peptides
df19 = MS2ORF.associate_evalues(df14, df17, 'MSC_masked', 'genome')
df19 = MS2ORF.associate_evalues(df19, df17, 'RC_masked', 'genome')
df19 = MS2ORF.associate_evalues(df19, df18, 'MSC_masked', 'proteome')
df19 = MS2ORF.associate_evalues(df19, df18, 'RC_masked', 'proteome')

#-----Notes:
#df19 contains main listing of features from database search, denovo sequencing, blast e-values, etc
#df17 contains tblastn results condensed down to a single entry for each unique peptide (by lowest e-value)
#df15 contains all tblastn results for comparing multiple hits
#df18 contains blastp results condensed down to a single entry for each unique peptides (by lowest e-value)

#-----output key dataframes:
df19.to_csv('processed_data/001_combined_df.csv', sep=',', index=False)
df17.to_csv('processed_data/001_tblastn_condensed_df.csv', sep=',', index=False)
df15.to_csv('processed_data/001_tblastn_expanded_df.csv', sep=',', index=False)

end = time.time()
print(end-start)

31.236429691314697


<H1>Gene sequence retrieval</H1>
<H3>finding gene sequences in the genome from peptide clues</H3>

In [2]:
start = time.time()

#-------------------------
#-----user input variables
#-------------------------

combined_dataframe1 = 'processed_data/001_combined_df.csv'
tblastn_condensed_dataframe1 = 'processed_data/001_tblastn_condensed_df.csv'
tblastn_expanded_dataframe1 = 'processed_data/001_tblastn_expanded_df.csv'
genome_fasta_file1 = 'raw_data/'
output_denovo_ORF_fasta1 = 'processed_data/001_putative_denovoIDs.fasta'

#-------------------------
#-------------------------

#-----read in csv files to dataframes
df19 = pd.read_csv(combined_dataframe1, sep=',')
df17 = pd.read_csv(tblastn_condensed_dataframe1, sep=',')
df15 = pd.read_csv(tblastn_expanded_dataframe1, sep=',')

#-----select rows from combined dataframe based on key parameters
#-----look for peptides that have low evalues when BLASTed against genome and no IDs when blasted against proteome/transcriptome
peptide_minlen = 5
genome_minevalue = 0
genome_maxevalue = 1
proteome_minevalue = -100
proteome_maxevalue = 0

df20 = df19.loc[(df19['sequence'] == '0') &
                ((df19['MSC_masked'].str.len() > peptide_minlen) & (df19['RC_masked'].str.len() > peptide_minlen)) &
                ((df19['MSC_masked_evalues_genome'] < genome_maxevalue) & (df19['MSC_masked_evalues_genome'] > genome_minevalue)) &
                ((df19['RC_masked_evalues_genome'] < genome_maxevalue) & (df19['RC_masked_evalues_genome'] > genome_minevalue)) &
                ((df19['MSC_masked_evalues_proteome'] < proteome_maxevalue) & (df19['MSC_masked_evalues_proteome'] > proteome_minevalue)) &
                ((df19['RC_masked_evalues_proteome'] < proteome_maxevalue) & (df19['RC_masked_evalues_proteome'] > proteome_minevalue)),
                ['scan', 'spectrum_ID', 'precursor_RC', 'charge_RC', 'peptide_score_MSC', 'peptide_score_RC', 'MSC_vs_RC', 'MSC_masked', 'RC_masked', 'MSC_masked_evalues_genome', 'RC_masked_evalues_genome', 'MSC_masked_evalues_proteome', 'RC_masked_evalues_proteome']
               ]

#-----extract peptides from combined dataframe
peptide_list1 = []
peptide_col_names1 = ['MSC_masked', 'RC_masked']
for item1 in peptide_col_names1:
    peptide_list1 += list(df20[item1])
peptide_list1 = list(set(peptide_list1))

#-----pare down tblastn dataframe to include only peptides selected in above steps
df21 = df15.loc[df15['sequence'].isin(peptide_list1)]

#-----select rows with evalues < genome_maxevalue (variable from above)
df21 = df21.loc[df21['evalue'] < genome_maxevalue].reset_index(drop=True)

#-----iterate through pared down tblastn dataframe to identify source genes and their start-stop boundaries within respective contigs
new_seq_list1 = []
new_start_list1 = []
new_stop_list1 = []

temp_new_seq = ''
temp_new_start = ''
temp_new_stop = ''

for row1 in df21.itertuples():
    temp_new_seq, temp_new_start, temp_new_stop = MS2ORF.extract_Gene_Sequence(genome_fasta_file1, row1.contig, row1.target_start, row1.target_stop)
    new_seq_list1.append(temp_new_seq)
    new_start_list1.append(temp_new_start)
    new_stop_list1.append(temp_new_stop)

df22 = df21.copy()
df22['gene_sequence'] = new_seq_list1
df22['target_gene_start'] = new_start_list1
df22['target_gene_stop'] = new_stop_list1

#-----save dataframe to .csv file
#df22.to_csv('001_isolated_genes_df.csv', index=False)

#-----create denovo protein fasta
MS2ORF.create_Denovo_Protein_Fasta(df22, 'gene_sequence', 'denovo_casanovo', output_denovo_ORF_fasta1)

end = time.time()
print(end-start)

/var/folders/b5/prg17fj579zd64gy82n9frm00000gn/T/ipykernel_51153/2105418884.py:17: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df19 = pd.read_csv(combined_dataframe1, sep=',')


7547.325511217117


<H1>Save Output to .csv</H1>

In [82]:
df22.to_csv('001_isolated_genes_df.csv', index=False)